In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import sys

# loading some functions we wrote before
sys.path.append("/Users/jonathanramos/Desktop/LRI/Image ROI Data Wrangling/")
from clean import *
from norm import *
from count import *

In [2]:
df = pd.read_csv('RAM-10,11_raw_data.csv')
print(df.columns)
print(df.shape)
df.head()

Index(['Unnamed: 0', ' ', 'AR', 'AnalysisDate', 'Area', 'Background',
       'CellNumber', 'Circ.', 'Experiment', 'FileName', 'Integrated_Density',
       'Max', 'Mean', 'Mean-Background', 'Min', 'Round', 'Solidity', 'Stain',
       'StdDev', 'Subject', 'XM', 'YM', 'directory', 'roiName', 'Rat_n',
       'Treatment', 'Dox', 'React', 'Coord'],
      dtype='object')
(70681, 29)


,Unnamed: 0,,AR,AnalysisDate,Area,Background,CellNumber,Circ.,Experiment,FileName,...,Subject,XM,YM,directory,roiName,Rat_n,Treatment,Dox,React,Coord
0,0,1,1.287,_11_20_2022_at_15-35,23.148,39.644,1,0.267,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,...,10C,30.322,16.611,/Volumes/T7/RAM-10/summed/double-label_results...,0020a-0035,RAM-10_rat10C,ChABC,DOX_ON,NR,"(30.322, 16.611)"
1,1,2,1.169,_11_20_2022_at_15-35,18.939,39.644,2,0.283,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,...,10C,141.285,29.432,/Volumes/T7/RAM-10/summed/double-label_results...,0036a-0169,RAM-10_rat10C,ChABC,DOX_ON,NR,"(141.285, 29.432)"
2,2,3,1.209,_11_20_2022_at_15-35,70.846,39.644,3,0.890,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,...,10C,54.411,393.511,/Volumes/T7/RAM-10/summed/double-label_results...,005a-00000,RAM-10_rat10C,ChABC,DOX_ON,NR,"(54.411, 393.511)"
3,3,4,1.000,_11_20_2022_at_15-35,67.338,39.644,4,1.000,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,...,10C,390.051,229.962,/Volumes/T7/RAM-10/summed/double-label_results...,005a-00001,RAM-10_rat10C,ChABC,DOX_ON,NR,"(390.051, 229.962)"
4,4,5,1.000,_11_20_2022_at_15-35,68.040,39.644,5,1.000,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,...,10C,211.344,302.819,/Volumes/T7/RAM-10/summed/double-label_results...,005a-00002,RAM-10_rat10C,ChABC,DOX_ON,NR,"(211.344, 302.819)"


# check for duplicates, nulls

In [3]:
print(df.duplicated(subset=['XM', 'YM', 'directory', 'roiName', 'FileName', 'Mean']).sum())

df.duplicated(subset=['XM', 'YM', 'directory', 'roiName', 'FileName', 'Mean'])

2471


0        False
1        False
2        False
3        False
4        False
         ...  
70676    False
70677    False
70678    False
70679    False
70680    False
Length: 70681, dtype: bool

In [4]:
df = df.drop_duplicates(subset=['XM', 'YM', 'directory', 'roiName', 'FileName', 'Mean'])
print(df.duplicated(subset=['XM', 'YM', 'directory', 'roiName', 'FileName', 'Mean']).sum())
print(df.shape)
df.head()

0
(68210, 29)


,Unnamed: 0,,AR,AnalysisDate,Area,Background,CellNumber,Circ.,Experiment,FileName,...,Subject,XM,YM,directory,roiName,Rat_n,Treatment,Dox,React,Coord
0,0,1,1.287,_11_20_2022_at_15-35,23.148,39.644,1,0.267,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,...,10C,30.322,16.611,/Volumes/T7/RAM-10/summed/double-label_results...,0020a-0035,RAM-10_rat10C,ChABC,DOX_ON,NR,"(30.322, 16.611)"
1,1,2,1.169,_11_20_2022_at_15-35,18.939,39.644,2,0.283,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,...,10C,141.285,29.432,/Volumes/T7/RAM-10/summed/double-label_results...,0036a-0169,RAM-10_rat10C,ChABC,DOX_ON,NR,"(141.285, 29.432)"
2,2,3,1.209,_11_20_2022_at_15-35,70.846,39.644,3,0.890,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,...,10C,54.411,393.511,/Volumes/T7/RAM-10/summed/double-label_results...,005a-00000,RAM-10_rat10C,ChABC,DOX_ON,NR,"(54.411, 393.511)"
3,3,4,1.000,_11_20_2022_at_15-35,67.338,39.644,4,1.000,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,...,10C,390.051,229.962,/Volumes/T7/RAM-10/summed/double-label_results...,005a-00001,RAM-10_rat10C,ChABC,DOX_ON,NR,"(390.051, 229.962)"
4,4,5,1.000,_11_20_2022_at_15-35,68.040,39.644,5,1.000,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,...,10C,211.344,302.819,/Volumes/T7/RAM-10/summed/double-label_results...,005a-00002,RAM-10_rat10C,ChABC,DOX_ON,NR,"(211.344, 302.819)"


In [5]:
# build cols we still need
df['mean-background'] = df.Mean - df.Background
df['treatment'] = df[['Dox', 'Treatment', 'React']].agg('_'.join, axis=1)
df['rat_n'] = df['Rat_n']
df.head()

,Unnamed: 0,,AR,AnalysisDate,Area,Background,CellNumber,Circ.,Experiment,FileName,...,directory,roiName,Rat_n,Treatment,Dox,React,Coord,mean-background,treatment,rat_n
0,0,1,1.287,_11_20_2022_at_15-35,23.148,39.644,1,0.267,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,...,/Volumes/T7/RAM-10/summed/double-label_results...,0020a-0035,RAM-10_rat10C,ChABC,DOX_ON,NR,"(30.322, 16.611)",10.776,DOX_ON_ChABC_NR,RAM-10_rat10C
1,1,2,1.169,_11_20_2022_at_15-35,18.939,39.644,2,0.283,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,...,/Volumes/T7/RAM-10/summed/double-label_results...,0036a-0169,RAM-10_rat10C,ChABC,DOX_ON,NR,"(141.285, 29.432)",15.218,DOX_ON_ChABC_NR,RAM-10_rat10C
2,2,3,1.209,_11_20_2022_at_15-35,70.846,39.644,3,0.890,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,...,/Volumes/T7/RAM-10/summed/double-label_results...,005a-00000,RAM-10_rat10C,ChABC,DOX_ON,NR,"(54.411, 393.511)",39.805,DOX_ON_ChABC_NR,RAM-10_rat10C
3,3,4,1.000,_11_20_2022_at_15-35,67.338,39.644,4,1.000,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,...,/Volumes/T7/RAM-10/summed/double-label_results...,005a-00001,RAM-10_rat10C,ChABC,DOX_ON,NR,"(390.051, 229.962)",64.620,DOX_ON_ChABC_NR,RAM-10_rat10C
4,4,5,1.000,_11_20_2022_at_15-35,68.040,39.644,5,1.000,RAM-10,RAM-10_10C_PFC_3.8_G_2.tif,...,/Volumes/T7/RAM-10/summed/double-label_results...,005a-00002,RAM-10_rat10C,ChABC,DOX_ON,NR,"(211.344, 302.819)",91.030,DOX_ON_ChABC_NR,RAM-10_rat10C


In [6]:
df.treatment.unique()

array(['DOX_ON_ChABC_NR', 'DOX_OFF_Vehicle_VR5', 'DOX_ON_ChABC_VR5',
       'DOX_ON_Vehicle_NR', 'DOX_OFF_Vehicle_NR', 'DOX_OFF_ChABC_FR1',
       'DOX_OFF_ChABC_VR5', 'DOX_ON_Vehicle_FR1', 'DOX_OFF_ChABC_NR',
       'DOX_OFF_Vehicle_FR1', 'DOX_ON_Vehicle_VR5', 'DOX_ON_ChABC_FR1'],
      dtype=object)

In [7]:
cFos_min = abs(df[df.Stain.str.contains('c-Fos')]['mean-background'].min())
mKate_min = abs(df[df.Stain.str.contains('mKate2')]['mean-background'].min())
EGFP_min = abs(df[df.Stain.str.contains('EGFP')]['mean-background'].min())

print(cFos_min)
print(mKate_min)
print(EGFP_min)

def adjust_intensity(x):
    if 'c-Fos' in x.Stain:
        return x['mean-background'] + cFos_min
    elif 'mKate2' in x.Stain:
        return x['mean-background'] + mKate_min
    elif 'EGFP' in x.Stain:
        return x['mean-background'] + EGFP_min
    else:
        print('missing val')
        return np.nan

df_adjusted = df.copy(deep=True)
df_adjusted['mean-background'] = df.apply(adjust_intensity, axis=1)


0.5420000000000016
2.263
0.7490000000000023


In [8]:
for stain in df.Stain.unique():
    print(stain)
    df_q = df_adjusted.query(f'Stain == "{stain}"').copy()
    norm_condition = 'DOX_OFF_Vehicle_FR1'
    df_norm_condition = df_q.query(f'treatment == "{norm_condition}"')
    norm_mean = df_norm_condition['mean-background'].mean()
    df_q['norm_mean-background'] = df_q['mean-background'] / norm_mean
    print(df_q.groupby('treatment')['norm_mean-background'].mean(),'\n')

    s = stain.replace('/','').replace('-','')

    prism_reorg(df_q)#.to_csv(f'RAM-10,11_norm_{s}_PRISM.csv')


c-Fos_coloc_w/_EGFP
treatment
DOX_OFF_ChABC_FR1      0.466849
DOX_OFF_ChABC_NR       0.820390
DOX_OFF_ChABC_VR5      0.400597
DOX_OFF_Vehicle_FR1    1.000000
DOX_OFF_Vehicle_NR     0.646372
DOX_OFF_Vehicle_VR5    0.577151
DOX_ON_ChABC_FR1       0.297097
DOX_ON_ChABC_NR        0.343914
DOX_ON_ChABC_VR5       0.522640
DOX_ON_Vehicle_FR1     0.479870
DOX_ON_Vehicle_NR      1.117638
DOX_ON_Vehicle_VR5     0.523877
Name: norm_mean-background, dtype: float64 

c-Fos_coloc_w/_mKate2
treatment
DOX_OFF_ChABC_FR1      0.444075
DOX_OFF_ChABC_NR       0.853819
DOX_OFF_ChABC_VR5      0.362796
DOX_OFF_Vehicle_FR1    1.000000
DOX_OFF_Vehicle_NR     0.467007
DOX_OFF_Vehicle_VR5    0.513567
DOX_ON_ChABC_FR1       0.268517
DOX_ON_ChABC_NR        0.350264
DOX_ON_ChABC_VR5       0.871428
DOX_ON_Vehicle_FR1     0.486026
DOX_ON_Vehicle_NR      0.879317
DOX_ON_Vehicle_VR5     0.483751
Name: norm_mean-background, dtype: float64 

EGFP_coloc_w/_c-Fos
treatment
DOX_OFF_ChABC_FR1      3.073313
DOX_OFF_ChABC_NR  

In [9]:
# confirming that no animals need to be removed due to low image ns
df_adjusted['image_name'] = df_adjusted.FileName.apply(lambda x: '_'.join(x.split('_')[:-1]))
q = df_adjusted.groupby('rat_n')['image_name'].unique().reset_index()
q['count'] = q.image_name.apply(len)
q

,rat_n,image_name,count
0,RAM-10_rat10C,"[RAM-10_10C_PFC_3.8_G, RAM-10_10C_PFC_3.9_E, R...",6
1,RAM-10_rat11C,"[RAM-10_11C_PFC_3.8_A, RAM-10_11C_PFC_3.8_D, R...",5
2,RAM-10_rat12C,"[RAM-10_12C_PFC_3.4_G, RAM-10_12C_PFC_3.5_E, R...",5
3,RAM-10_rat13C,"[RAM-10_13C_PFC_3.7_D, RAM-10_13C_PFC_3.9_B, R...",4
4,RAM-10_rat14C,"[RAM-10_14C_PFC_3.7_H, RAM-10_14C_PFC_3.8_F, R...",5
5,RAM-10_rat15C,"[RAM-10_15C_PFC_3.6_F, RAM-10_15C_PFC_3.7_E, R...",4
6,RAM-10_rat16C,"[RAM-10_16C_PFC_3.6_A, RAM-10_16C_PFC_3.6_D, R...",5
7,RAM-10_rat1C,"[RAM-10_1C_PFC_3.5_D, RAM-10_1C_PFC_3.7_C, RAM...",4
8,RAM-10_rat2C,"[RAM-10_2C_PFC_3.6_C, RAM-10_2C_PFC_3.7_B, RAM...",5
9,RAM-10_rat3C,"[RAM-10_3C_PFC_3.8_D, RAM-10_3C_PFC_3.9_B, RAM...",5


In [10]:
df_adjusted['stain'] = df_adjusted.Stain.str.replace('/','')
df_adjusted['react'] = df_adjusted.React
df_adjusted['dox'] = df_adjusted.Dox

df_adjusted = df_adjusted.drop(['Unnamed: 0', ' ', 'AR', 'AnalysisDate', 'Area',
       'CellNumber', 'Circ.', 'Experiment', 'Integrated_Density',
       'Max', 'Mean-Background', 'Min', 'Round', 'Solidity', 'Stain',
       'StdDev', 'Subject', 'directory', 'roiName', 'Rat_n',
       'Treatment', 'Dox', 'React'], axis=1).copy(deep=True)

print(df_adjusted.columns)
print(df_adjusted.shape)
df_adjusted.head()

Index(['Background', 'FileName', 'Mean', 'XM', 'YM', 'Coord',
       'mean-background', 'treatment', 'rat_n', 'image_name', 'stain', 'react',
       'dox'],
      dtype='object')
(68210, 13)


,Background,FileName,Mean,XM,YM,Coord,mean-background,treatment,rat_n,image_name,stain,react,dox
0,39.644,RAM-10_10C_PFC_3.8_G_2.tif,50.420,30.322,16.611,"(30.322, 16.611)",11.318,DOX_ON_ChABC_NR,RAM-10_rat10C,RAM-10_10C_PFC_3.8_G,c-Fos_coloc_w_EGFP,NR,DOX_ON
1,39.644,RAM-10_10C_PFC_3.8_G_2.tif,54.862,141.285,29.432,"(141.285, 29.432)",15.760,DOX_ON_ChABC_NR,RAM-10_rat10C,RAM-10_10C_PFC_3.8_G,c-Fos_coloc_w_EGFP,NR,DOX_ON
2,39.644,RAM-10_10C_PFC_3.8_G_2.tif,79.449,54.411,393.511,"(54.411, 393.511)",40.347,DOX_ON_ChABC_NR,RAM-10_rat10C,RAM-10_10C_PFC_3.8_G,c-Fos_coloc_w_EGFP,NR,DOX_ON
3,39.644,RAM-10_10C_PFC_3.8_G_2.tif,104.264,390.051,229.962,"(390.051, 229.962)",65.162,DOX_ON_ChABC_NR,RAM-10_rat10C,RAM-10_10C_PFC_3.8_G,c-Fos_coloc_w_EGFP,NR,DOX_ON
4,39.644,RAM-10_10C_PFC_3.8_G_2.tif,130.674,211.344,302.819,"(211.344, 302.819)",91.572,DOX_ON_ChABC_NR,RAM-10_rat10C,RAM-10_10C_PFC_3.8_G,c-Fos_coloc_w_EGFP,NR,DOX_ON


# Counting Mean Cell ns

In [13]:
df_img_ns = df_adjusted.groupby('rat_n')['image_name'].unique().reset_index()
df_img_ns['n_imgs'] = df_img_ns.image_name.apply(len)
df_img_ns

,rat_n,image_name,n_imgs
0,RAM-10_rat10C,"[RAM-10_10C_PFC_3.8_G, RAM-10_10C_PFC_3.9_E, R...",6
1,RAM-10_rat11C,"[RAM-10_11C_PFC_3.8_A, RAM-10_11C_PFC_3.8_D, R...",5
2,RAM-10_rat12C,"[RAM-10_12C_PFC_3.4_G, RAM-10_12C_PFC_3.5_E, R...",5
3,RAM-10_rat13C,"[RAM-10_13C_PFC_3.7_D, RAM-10_13C_PFC_3.9_B, R...",4
4,RAM-10_rat14C,"[RAM-10_14C_PFC_3.7_H, RAM-10_14C_PFC_3.8_F, R...",5
5,RAM-10_rat15C,"[RAM-10_15C_PFC_3.6_F, RAM-10_15C_PFC_3.7_E, R...",4
6,RAM-10_rat16C,"[RAM-10_16C_PFC_3.6_A, RAM-10_16C_PFC_3.6_D, R...",5
7,RAM-10_rat1C,"[RAM-10_1C_PFC_3.5_D, RAM-10_1C_PFC_3.7_C, RAM...",4
8,RAM-10_rat2C,"[RAM-10_2C_PFC_3.6_C, RAM-10_2C_PFC_3.7_B, RAM...",5
9,RAM-10_rat3C,"[RAM-10_3C_PFC_3.8_D, RAM-10_3C_PFC_3.9_B, RAM...",5


## all stain-type combinations

In [47]:
df_adjusted['stain'] = df_adjusted.stain.str.replace('-','')
def mean_cell_ns_single(stain_x):
    # series of groupby's and reductions to get sum of sums per rat
    # first groupby to count cells per image; second groupby to sum counts per rat
    # merge with df to get required labels, merge with df_img_ns to get n_imgs per rat
    q = df_adjusted.query(f'stain == "{stain_x}"')\
        .groupby('image_name')['rat_n'].count()\
        .reset_index().rename(columns={'rat_n':'count'})\
        .merge(df_adjusted[['image_name', 'rat_n']], on='image_name', how='left')\
        .drop_duplicates()\
        .groupby('rat_n')['count'].sum().reset_index().rename(columns={'count':'sum'})\
        .merge(df_adjusted[['rat_n', 'react', 'dox']],\
            on='rat_n', how='left')\
        .drop_duplicates().reset_index()\
        .merge(df_img_ns[['rat_n', 'n_imgs']], on='rat_n', how='left')

    # compute means
    q['mean_cell_n'] = q['sum'] / q['n_imgs']

    # sort by dox then react; reorder cols
    q = q.sort_values(by=['dox', 'react'])\
        [['rat_n', 'dox', 'react', 'sum', 'n_imgs', 'mean_cell_n']]\
        .reset_index().drop('index', axis=1)
    
    q = q.merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right')

    return q

df_total_cFos = mean_cell_ns_single('cFos')
df_total_mKate = mean_cell_ns_single('mKate2')

for stain in df_adjusted.stain.unique():
    print(stain)
    mean_cell_ns_single(stain).to_csv(f'RAM-10,11_{stain}_mean_cell_ns.csv')

df_total_cFos_on_mKate = mean_cell_ns_single('cFos_coloc_w_mKate2')
df_total_mKate_on_cFos = mean_cell_ns_single('mKate2_coloc_w_cFos')

cFos_coloc_w_EGFP
cFos_coloc_w_mKate2
EGFP_coloc_w_cFos
EGFP_coloc_w_mKate2
mKate2_coloc_w_cFos
mKate2_coloc_w_EGFP
cFos
EGFP
mKate2
Triplelabeled_cFos
Triplelabeled_EGFP
Triplelabeled_mKate2


adding a new cell type col to identify cells by their primary stain type (i.e. computer will get confused if i ask it "give me all cells whose stain type contains 'cFos'"; in this case it will return cFos_coloc_mKate2 as well as mKate2_coloc_w_cFos when we really only want the former)

In [38]:
df_adjusted['cell_type'] = df_adjusted.stain.apply(\
    lambda x: x.split('_')[0] if not 'Triple' in x else x.split('_')[-1])

df_adjusted.cell_type.unique()


array(['cFos', 'EGFP', 'mKate2'], dtype=object)

## splitting hi/lo cFos

In [39]:
# compute median from all cFos cells
df_cFos = df_adjusted.query('stain == "cFos"').copy()
cFos_median = df_cFos['mean-background'].median()

# apply median split across all cFos stain types
df_cFos = df_adjusted.query('cell_type == "cFos"').copy()
df_cFos['cFos_split'] = df_cFos['mean-background'].apply(lambda x: 'cFos_hi' if x > cFos_median else 'cFos_lo')

### hi/lo cFos counts

In [45]:
def mean_cell_ns_cFosSplit(cFos_split, stain_x, df_key):
    # series of groupby's and reductions to get sum of sums per rat
    # first groupby to count cells per image; second groupby to sum counts per rat
    # merge with df to get required labels, merge with df_img_ns to get n_imgs per rat
    q = df_cFos.query(f'cFos_split == "{cFos_split}" and stain == "{stain_x}"')\
        .groupby('image_name')['rat_n'].count()\
        .reset_index().rename(columns={'rat_n':'count'})\
        .merge(df_adjusted[['image_name', 'rat_n']], on='image_name', how='left')\
        .drop_duplicates()\
        .groupby('rat_n')['count'].sum().reset_index().rename(columns={'count':'sum'})\
        .merge(df_adjusted[['rat_n', 'react', 'dox']],\
            on='rat_n', how='left')\
        .drop_duplicates().reset_index()\
        .merge(df_img_ns[['rat_n', 'n_imgs']], on='rat_n', how='left')

    # compute means
    q['mean_cell_n'] = q['sum'] / q['n_imgs']

    # sort by dox then react; reorder cols
    q = q.sort_values(by=['dox', 'react'])\
        [['rat_n', 'dox', 'react', 'sum', 'n_imgs', 'mean_cell_n']]\
        .reset_index().drop('index', axis=1)

    # merge with key to ensure rats without specified stain type are represented with NaN
    q = q.merge(df_key, on=list(df_key.columns), how='right')

    return q


df_key = df_total_cFos[['rat_n', 'dox', 'react']].copy(deep=True)
mean_cell_ns_cFosSplit('cFos_hi', 'cFos', df_key).to_csv('RAM-10,11_cFos_hi_mean_cell_ns.csv')
mean_cell_ns_cFosSplit('cFos_lo', 'cFos', df_key).to_csv('RAM-10,11_cFos_lo_mean_cell_ns.csv')

mean_cell_ns_cFosSplit('cFos_hi', 'cFos_coloc_w_mKate2', df_key).to_csv('RAM-10,11_cFos_hi_coloc_w_mKate2_mean_cell_ns.csv')
mean_cell_ns_cFosSplit('cFos_lo', 'cFos_coloc_w_mKate2', df_key).to_csv('RAM-10,11_cFos_lo_coloc_w_mKate2_mean_cell_ns.csv')
mean_cell_ns_cFosSplit('cFos_hi', 'mKate2_coloc_w_cFos', df_key).to_csv('RAM-10,11_mKate2_coloc_w_cFos_hi_mean_cell_ns.csv')
mean_cell_ns_cFosSplit('cFos_lo', 'mKate2_coloc_w_cFos', df_key).to_csv('RAM-10,11_mKate2_coloc_w_cFos_lo_mean_cell_ns.csv')

## Percent overlap

### total cFos/mKate percent overlap

In [49]:
df_total_cFos_on_mKate['total_cFos_on_mKate'] = df_total_cFos_on_mKate['sum']
df_total_mKate_on_cFos['total_mKate_on_cFos'] = df_total_mKate_on_cFos['sum']

df_total_cFos['total_cFos'] = df_total_cFos['sum']
df_total_mKate['total_mKate'] = df_total_mKate['sum']
df_percent_cFos_on_mKate = df_total_cFos_on_mKate[['rat_n', 'dox', 'react', 'total_cFos_on_mKate']].merge(df_total_cFos[['rat_n', 'dox', 'react', 'total_cFos']], on=['rat_n', 'dox', 'react'], how='left')
df_percent_cFos_on_mKate['percent'] = df_percent_cFos_on_mKate.total_cFos_on_mKate / df_percent_cFos_on_mKate.total_cFos * 100
df_percent_cFos_on_mKate.merge(df_total_mKate[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv('RAM-10,11_TOTAL_cFos_percent_mKate_overlap.csv')

df_percent_mKate_on_cFos = df_total_mKate_on_cFos[['rat_n', 'dox', 'react', 'total_mKate_on_cFos']].merge(df_total_mKate[['rat_n', 'dox', 'react', 'total_mKate']], on=['rat_n', 'dox', 'react'], how='left')
df_percent_mKate_on_cFos['percent'] = df_percent_mKate_on_cFos.total_mKate_on_cFos / df_percent_mKate_on_cFos.total_mKate * 100
df_percent_mKate_on_cFos.merge(df_total_cFos[['rat_n', 'dox', 'react']], on=['rat_n','dox', 'react'], how='right').to_csv('RAM-10,11_TOTAL_mKate_percent_cFos_overlap.csv')

### cFos hi

In [52]:
df_total_cFos_hi = mean_cell_ns_cFosSplit('cFos_hi', 'cFos', df_key)
df_total_cFos_lo = mean_cell_ns_cFosSplit('cFos_lo', 'cFos', df_key)


df_cFos_hi_coloc_w_mKate2 = mean_cell_ns_cFosSplit('cFos_hi', 'cFos_coloc_w_mKate2', df_key)
df_cFos_lo_coloc_w_mKate2 = mean_cell_ns_cFosSplit('cFos_lo', 'cFos_coloc_w_mKate2', df_key)


,rat_n,dox,react,sum,n_imgs,mean_cell_n,total_mKate
0,RAM-10_rat15C,DOX_OFF,FR1,118,4,29.500000,118
1,RAM-10_rat5C,DOX_OFF,FR1,257,6,42.833333,257
2,RAM-10_rat7C,DOX_OFF,FR1,116,6,19.333333,116
3,RAM-11_rat14,DOX_OFF,FR1,219,6,36.500000,219
4,RAM-11_rat15,DOX_OFF,FR1,258,6,43.000000,258
5,RAM-11_rat3,DOX_OFF,FR1,179,6,29.833333,179
6,RAM-11_rat5,DOX_OFF,FR1,140,6,23.333333,140
7,RAM-10_rat14C,DOX_OFF,NR,20,5,4.000000,20
8,RAM-10_rat3C,DOX_OFF,NR,18,5,3.600000,18
9,RAM-11_rat11,DOX_OFF,NR,158,6,26.333333,158


In [54]:
df_mKate2_coloc_w_cFos_hi = mean_cell_ns_cFosSplit('cFos_hi', 'mKate2_coloc_w_cFos', df_key)
df_mKate2_coloc_w_cFos_lo

,rat_n,dox,react,sum,n_imgs,mean_cell_n
0,RAM-10_rat15C,DOX_OFF,FR1,NaN,NaN,NaN
1,RAM-10_rat5C,DOX_OFF,FR1,NaN,NaN,NaN
2,RAM-10_rat7C,DOX_OFF,FR1,NaN,NaN,NaN
3,RAM-11_rat14,DOX_OFF,FR1,NaN,NaN,NaN
4,RAM-11_rat15,DOX_OFF,FR1,NaN,NaN,NaN
5,RAM-11_rat3,DOX_OFF,FR1,NaN,NaN,NaN
6,RAM-11_rat5,DOX_OFF,FR1,NaN,NaN,NaN
7,RAM-10_rat14C,DOX_OFF,NR,NaN,NaN,NaN
8,RAM-10_rat3C,DOX_OFF,NR,NaN,NaN,NaN
9,RAM-11_rat11,DOX_OFF,NR,NaN,NaN,NaN
